-> SQL Alchemy

-> psycopg2

-> pandas.io.psql

In [ ]:
# pip install psycopg2-binary

In [1]:
import psycopg2 as pg
import pandas as pd

In [2]:
# Database: Postgress 12.0 - AWS
# credenciais
host = 'comunidade-ds-postgres.c50pcakiuwi3.us-east-1.rds.amazonaws.com'
port = 5432
database = 'comunidadedsdb'
username = 'member'
password = 'cdspa'

In [3]:
# conectando ao DB
conn = pg.connect( user = username,
                   password = password,
                   host = host,
                   port = port,
                   database = database)

cursor = conn.cursor()



**Query que mostra as Schemas do Database**

In [ ]:
query_schema = """
    SELECT nspname
    FROM pg_catalog.pg_namespace
"""

cursor.execute(query_schema)

record = cursor.fetchall()
cursor.close()
conn.close()

print(record)

**Query que mostra as tabelas dentro da Schema PA004**

In [ ]:
query_table = """
    SELECT tablename
    FROM pg_tables
    WHERE schemaname='pa004'
"""

cursor.execute(query_table)

record = cursor.fetchall()
cursor.close()
conn.close()

print(record)

In [ ]:
query_test = """
    SELECT *
    FROM pa004.users u
    LIMIT 10
"""

cursor.execute(query_test)

record = cursor.fetchall()
cursor.close()
conn.close()

print(record)

In [ ]:
data = pd.DataFrame( record )
data

Como é possivel observar, ao se transformar em DataFrame não tem as colunas nomeadas. 

### Coletando dados Com Pandas

In [ ]:
import pandas.io.sql as psql

In [10]:
query_tables = """
    SELECT *
    FROM pa004.users u LEFT JOIN pa004.vehicle v ON (u.id = v.id)
                       LEFT JOIN pa004.insurance i ON (u.id = i.id)
   LIMIT 10
"""

cursor.execute(query_tables)

df_raw = pd.read_sql(query_tables, conn) # não precisa de cursor, faz a conexão direto
# df_raw = df_raw.drop(df_raw.columns[[5,9]], axis=1)
df_raw.head()

/tmp/ipykernel_10365/648916041.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_raw = pd.read_sql(query_tables, conn) # não precisa de cursor, faz a conexão direto


,id,gender,age,region_code,policy_sales_channel,id,driving_license,vehicle_age,vehicle_damage,id,previously_insured,annual_premium,vintage,response
0,9531,Male,22,45.0,152.0,9531,1,< 1 Year,No,9531,1,25120.0,289,0
1,9548,Female,23,46.0,152.0,9548,1,< 1 Year,Yes,9548,0,30899.0,28,0
2,9553,Male,44,9.0,124.0,9553,1,> 2 Years,Yes,9553,0,34372.0,73,1
3,9554,Female,49,28.0,26.0,9554,1,1-2 Year,No,9554,1,27626.0,36,0
4,9567,Male,75,31.0,9.0,9567,1,1-2 Year,Yes,9567,1,2630.0,151,0


In [23]:
df1 = df_raw.copy()

In [24]:
df1 = df1.loc[:, ~df1.columns.duplicated()]
df1.head()

,id,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,9531,Male,22,45.0,152.0,1,< 1 Year,No,1,25120.0,289,0
1,9548,Female,23,46.0,152.0,1,< 1 Year,Yes,0,30899.0,28,0
2,9553,Male,44,9.0,124.0,1,> 2 Years,Yes,0,34372.0,73,1
3,9554,Female,49,28.0,26.0,1,1-2 Year,No,1,27626.0,36,0
4,9567,Male,75,31.0,9.0,1,1-2 Year,Yes,1,2630.0,151,0


In [22]:
df1 = df1.T.drop_duplicates().T # para excluir colunas duplicadas
df1.head()

,id,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,9531,Male,22,45.0,152.0,1,< 1 Year,No,1,25120.0,289,0
1,9548,Female,23,46.0,152.0,1,< 1 Year,Yes,0,30899.0,28,0
2,9553,Male,44,9.0,124.0,1,> 2 Years,Yes,0,34372.0,73,1
3,9554,Female,49,28.0,26.0,1,1-2 Year,No,1,27626.0,36,0
4,9567,Male,75,31.0,9.0,1,1-2 Year,Yes,1,2630.0,151,0


### SQL Alchemy